# Preparação do ambiente

## Bibliotecas

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import unidecode
import warnings 

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from string import punctuation

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore')

## Constantes e funções auxiliares

In [3]:
def tokenizer(doc):
    doc = doc.lower()
    doc = unidecode.unidecode(doc)
    
    patterns = [('wi\s?.?fi', 'wifi'), 
                ('2,4', '2.4'), 
                ('5,8', '5.8')]
    for pattern, repl in patterns:
        doc = re.sub(pattern, repl, doc)
    
    stop_words = stopwords.words('portuguese')
    stop_words.extend(stopwords.words('english'))
    stop_words.extend(list(punctuation))
    # stopwords específicas do domínio
    stop_words.extend(['cm', 'feature', 'features', 'informações', 'itens', 'leve', 'list', 'nulo', 'package', 'pacote', 'pacotes', 'recurso', 'tamanho', 'ver', 'unidades', 'fio', 'universal'])
    # cores mais comumns
    stop_words.extend(['preto', 'cinza', 'branco', 'rosa', 'vermelho', 'laranja', 'amarelo', 'verde', 'azul', 'roxo', 'marrom'])
    # remover da lista de stopwords a palavra sem para formar o bigrama "sem fio", que pode ser relevante para o domínio
    # stop_words.remove('sem') 
    
    tokens = [token for token in word_tokenize(doc) if token not in stop_words]
    return tokens

def doc_cleaner(doc):
    tokens = tokenizer(doc)
    return ' '.join(tokens) 

# Carga e prepação dos dados

In [4]:
file_cat_sample = '../datasets/cat_to_label_sample.xlsx'
df_cat_sample = pd.read_excel(file_cat_sample)

df_cat_sample = df_cat_sample.iloc[:,-2:]
df_cat_sample = df_cat_sample.dropna()
df_cat_sample.columns = ['doc', 'labeled_category']

map_telecom_product = {'Antena': 1,  
                       'Baterias (eletrônicos em geral)': 0, 
                       'Cabo (em geral)': 0, 
                       'Cabo de rede': 1, 
                       'Câmera sem fio': 1, 
                       'Carregador de celular': 1,
                       'Carregador portátil (powerbank)': 1,
                       'Carregador sem fio': 1, 
                       'Carregadores e fontes (eletrônicos em geral)': 0, 
                       'Cartão de memória': 0,
                       'Desktop/Notebook': 0, 
                       'Drone': 1, 
                       'Extensor de vídeo VGA/HDMI': 0, 
                       'Fone de ouvido sem fio': 1, 
                       'Hub Switch': 1, 
                       'Impressora e cartuchos': 0,
                       'Microfone sem fio': 1, 
                       'Modem 3G/4G/WiFi': 1, 
                       'Outros': 0, 
                       'Reforçador de sinal de celular': 1, 
                       'Roteador/Reforçador WiFi': 1, 
                       'Smart TV': 1,
                       'Smartwatch': 1, 
                       'Tablets': 1, 
                       'Telefone celular': 1, 
                       'Transceptor de Radiação Restrita': 1, 
                       'Transceptor de RF (HF/VHF/UHF)': 1, 
                       'Transceptor/Conversor SFP': 1, 
                       'TV Box': 1}

map_rf_transmitter = {'Antena': 1,  
                      'Baterias (eletrônicos em geral)': 0, 
                      'Cabo (em geral)': 0, 
                      'Cabo de rede': 1, 
                      'Câmera sem fio': 2, 
                      'Carregador de celular': 1,
                      'Carregador portátil (powerbank)': 1,
                      'Carregador sem fio': 1, 
                      'Carregadores e fontes (eletrônicos em geral)': 0, 
                      'Cartão de memória': 0,
                      'Desktop/Notebook': 0, 
                      'Drone': 2, 
                      'Extensor de vídeo VGA/HDMI': 0, 
                      'Fone de ouvido sem fio': 2, 
                      'Hub Switch': 1, 
                      'Impressora e cartuchos': 0,
                      'Microfone sem fio': 2, 
                      'Modem 3G/4G/WiFi': 2, 
                      'Outros': 0,                        
                      'Reforçador de sinal de celular': 2, 
                      'Roteador/Reforçador WiFi': 2, 
                      'Smart TV': 2,
                      'Smartwatch': 2, 
                      'Tablets': 2, 
                      'Telefone celular': 2, 
                      'Transceptor de Radiação Restrita': 2, 
                      'Transceptor de RF (HF/VHF/UHF)': 2, 
                      'Transceptor/Conversor SFP': 1, 
                      'TV Box': 2}

df_cat_sample['telecom_product'] = df_cat_sample['labeled_category'].map(map_telecom_product)
df_cat_sample['rf_transmitter'] = df_cat_sample['labeled_category'].map(map_rf_transmitter)

map_category_to_int = {category:i for i,category in enumerate(df_cat_sample.labeled_category.unique())}
map_category_to_name = {i:category for category,i in map_category_to_int.items()}
df_cat_sample['category'] = df_cat_sample['labeled_category'].map(map_category_to_int)

df_cat_sample['clean_doc'] = df_cat_sample.doc.map(doc_cleaner)
columns_to_keep = ['doc', 'clean_doc', 'labeled_category', 'telecom_product', 'rf_transmitter', 'category']
df_cat_sample = df_cat_sample[columns_to_keep]

df_cat_sample

,doc,clean_doc,labeled_category,telecom_product,rf_transmitter,category
0,"Antena WiFi omnidirecional 2 peças, antena 4DB...",antena wifi omnidirecional 2 pecas antena 4dbi...,Antena,1,1,0
1,"Kesoto 2 peças macho 2,4 GHz 5 Ghz 5,8 Ghz ant...",kesoto 2 pecas macho 2.4 ghz 5 ghz 5.8 ghz ant...,Antena,1,1,0
2,Hemobllo Antenas 5 Unidades módulo transceptor...,hemobllo antenas 5 modulo transceptor antena b...,Antena,1,1,0
3,3 peças placa de rede sem fio WiFi antena rote...,3 pecas placa rede wifi antena roteador 2.4g/5...,Antena,1,1,0
4,Angoily 2 Unidades Substituição Da Antena Bast...,angoily 2 substituicao antena bastoes cola rec...,Antena,1,1,0
...,...,...,...,...,...,...
2086,Gps para Bike Xoss G Bluetooth Mtb Speed,gps bike xoss g bluetooth mtb speed,Transceptor de Radiação Restrita,1,2,25
2087,"Módulo AUX Bluetooth para carro, 11Pin Bluetoo...",modulo aux bluetooth carro 11pin bluetooth 5.0...,Transceptor de Radiação Restrita,1,2,25
2088,Operitacx Batente De Porta Protetor De Parada ...,operitacx batente porta protetor parada piso p...,Outros,0,0,18
2089,"Tecido de Faraday de 43 x 39 polegadas, tecido...",tecido faraday 43 x 39 polegadas tecido farada...,Outros,0,0,18


# Análise

## Classificação binária

In [5]:
X = df_cat_sample['doc']
y = df_cat_sample['telecom_product']
target_names = ['Produtos comuns', 'Produtos de telecomunicações']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=724)

In [6]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
clf_dummy = DummyClassifier()
clf_dummy.fit(X_train_tfidf,y_train)
predicted = clf_dummy.predict(X_test_tfidf)

print('Accuracy of Dummy classifier on training set: {:.2f}'
     .format(clf_dummy.score(X_train, y_train)))
print('Accuracy of Dummy classifier on test set: {:.2f}'
     .format(clf_dummy.score(X_test, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of Dummy classifier on training set: 0.55
Accuracy of Dummy classifier on test set: 0.58
                              precision    recall  f1-score   support

             Produtos comuns       0.58      1.00      0.73       303
Produtos de telecomunicações       0.00      0.00      0.00       220

                    accuracy                           0.58       523
                   macro avg       0.29      0.50      0.37       523
                weighted avg       0.34      0.58      0.43       523



In [8]:
clf_sgd = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)

clf_sgd.fit(X_train_tfidf, y_train)
predicted = clf_sgd.predict(X_test_tfidf)

print('Accuracy of SGD classifier on training set: {:.2f}'
     .format(clf_sgd.score(X_train_tfidf, y_train)))
print('Accuracy of SGD classifier on test set: {:.2f}'
     .format(clf_sgd.score(X_test_tfidf, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.96
Accuracy of SGD classifier on test set: 0.88
                              precision    recall  f1-score   support

             Produtos comuns       0.88      0.90      0.89       303
Produtos de telecomunicações       0.86      0.84      0.85       220

                    accuracy                           0.88       523
                   macro avg       0.87      0.87      0.87       523
                weighted avg       0.88      0.88      0.88       523



## Classificação em 3 classes

In [9]:
X = df_cat_sample['doc']
y = df_cat_sample['rf_transmitter']
target_names = ['Produtos comuns', 'Produtos de telecomunicações', 'Transmissores de RF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=724)

In [10]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [11]:
clf_dummy = DummyClassifier()
clf_dummy.fit(X_train_tfidf,y_train)
predicted = clf_dummy.predict(X_test_tfidf)

print('Accuracy of Dummy classifier on training set: {:.2f}'
     .format(clf_dummy.score(X_train_tfidf, y_train)))
print('Accuracy of Dummy classifier on test set: {:.2f}'
     .format(clf_dummy.score(X_test_tfidf, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of Dummy classifier on training set: 0.55
Accuracy of Dummy classifier on test set: 0.58
                              precision    recall  f1-score   support

             Produtos comuns       0.58      1.00      0.73       303
Produtos de telecomunicações       0.00      0.00      0.00        28
         Transmissores de RF       0.00      0.00      0.00       192

                    accuracy                           0.58       523
                   macro avg       0.19      0.33      0.24       523
                weighted avg       0.34      0.58      0.43       523



In [12]:
clf_sgd = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)

clf_sgd.fit(X_train_tfidf, y_train)
predicted = clf_sgd.predict(X_test_tfidf)

print('Accuracy of SGD classifier on training set: {:.2f}'
     .format(clf_sgd.score(X_train_tfidf, y_train)))
print('Accuracy of SGD classifier on test set: {:.2f}'
     .format(clf_sgd.score(X_test_tfidf, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.96
Accuracy of SGD classifier on test set: 0.87
                              precision    recall  f1-score   support

             Produtos comuns       0.87      0.92      0.89       303
Produtos de telecomunicações       1.00      0.21      0.35        28
         Transmissores de RF       0.86      0.89      0.88       192

                    accuracy                           0.87       523
                   macro avg       0.91      0.67      0.71       523
                weighted avg       0.88      0.87      0.86       523

